In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
from matplotlib import pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 500)

<h1> Known features and components </h1>

<h3> IMX304 Sensor Data Sheet: https://en.ids-imaging.com/sony-imx304.html </h3>

<h1> Generate results table </h1>

In [ ]:
def generate_case_results(focal_length, r_s, camera_sensor_size_horizontal, camera_sensor_size_vertical, working_distance, baseline, fish_length, fish_width):
    horizontal_field_of_view = 2 * np.arctan(camera_sensor_size_horizontal / (2.0 * focal_length))
    vertical_field_of_view = 2 * np.arctan(camera_sensor_size_vertical / (2.0 * focal_length))

    # get estimated errors in estimated fish total length
    fish_pixel_length = fish_length * (focal_length / working_distance)

    # get estimated errors in estimated fish width
    fish_pixel_width = fish_width * (focal_length / working_distance)

    # get horiontal metrics
    field_of_view_total_size_horizontal = 2 * working_distance * np.tan(horizontal_field_of_view / 2)
    overlapping_region_size_horizontal = max(0, field_of_view_total_size_horizontal - baseline)
    fish_fully_in_overlapping_region_proability_horizontal = (overlapping_region_size_horizontal - fish_length) / (field_of_view_total_size_horizontal - fish_length)
    resolution = r_s * focal_length / working_distance
    
#     overlapping_region_size_in_fish_horizontal = overlapping_region_size_horizontal / float(fish_length)
#     overlapping_region_fraction_horizontal = overlapping_region_size_horizontal / float(field_of_view_total_size_horizontal)
#     fish_size_fraction_horizontal = fish_length / float(field_of_view_total_size_horizontal)
    
    # get vertical metrics
    field_of_view_total_size_vertical = 2 * working_distance * np.tan(vertical_field_of_view / 2)
    fish_size_fraction_vertical = fish_length / field_of_view_total_size_vertical
    
    
    # generate results hash
    results = {
        'focal_length': focal_length,
        'camera_sensor_size_horizontal': camera_sensor_size_horizontal,
        'camera_sensor_size_vertical': camera_sensor_size_vertical,
        'baseline': baseline, 
        'working_distance': working_distance,
        'fish_length': fish_length,
        'fish_width': fish_width,
        'overlapping_region_size_horizontal': overlapping_region_size_horizontal,
#         'overlapping_region_size_in_fish_horizontal': overlapping_region_size_in_fish_horizontal,
#         'overlapping_region_fraction_horizontal': overlapping_region_fraction_horizontal,
#         'fish_size_fraction_horizontal': fish_size_fraction_horizontal
        'fish_fully_in_overlapping_region_proability_horizontal': fish_fully_in_overlapping_region_proability_horizontal if fish_fully_in_overlapping_region_proability_horizontal > 0 else np.nan,
        'fish_size_fraction_vertical': fish_size_fraction_vertical,
        'horizontal_field_of_view': horizontal_field_of_view * (180 / np.pi),
        'vertical_field_of_view': vertical_field_of_view * (180 / np.pi),
        'resolution': resolution
    }

    return results
    

In [ ]:
px_count_horizontal = 4096
px_count_vertical = 3000
camera_sensor_size_horizontal = 1.412  
camera_sensor_size_vertical = 1.034
r_s = 125
effective_pixel_size_horizontal = camera_sensor_size_horizontal / px_count_horizontal
effective_pixel_size_vertical = camera_sensor_size_horizontal / px_count_vertical


In [ ]:
# note: all values are in centimeters unless specified otherwise

focal_lengths = np.arange(0.8, 2.0, 0.1)
fish_length_list = list(np.arange(40, 90, 10))
fish_width_list = list(np.arange(15, 30, 5))
working_distance_list = list(np.arange(50, 500, 50))
baselines = [round(baseline, 1) for baseline in np.arange(10.0, 25.0, 1.0)]

results_lists = defaultdict(list)

for focal_length in focal_lengths:
    for fish_length in fish_length_list:
        for fish_width in fish_width_list:
            for working_distance in working_distance_list:
                for baseline in baselines:
                    results = generate_case_results(focal_length, r_s, camera_sensor_size_horizontal, camera_sensor_size_vertical, working_distance, baseline, fish_length, fish_width)
                    for key, val in results.iteritems():
                        results_lists[key].append(val)

results_df = pd.DataFrame(results_lists)

In [ ]:
mask = (results_df.working_distance == 100) & (results_df.fish_length == 80) & (results_df.baseline == 13) & (~results_df.isnull().any(axis=1))
results_df[mask].sort_values('overlapping_region_size_horizontal', ascending=False)

In [ ]:
fish_length = 80
baseline = 13
plt.figure(figsize=(20, 10))
for working_distance in [100, 200, 300, 400, 500]:
    mask = (results_df.working_distance == working_distance) & (results_df.fish_length == fish_length) & (results_df.baseline == baseline) & (~results_df.isnull().any(axis=1))
    tdf = results_df[mask].sort_values('overlapping_region_size_horizontal', ascending=False).copy(deep=True)
    plt.plot(tdf.horizontal_field_of_view, tdf.overlapping_region_size_horizontal, label=working_distance)
    
plt.legend()
plt.xlabel('Field of View (degrees)')
plt.ylabel('Size of overlapping region (cm)')
plt.title('Size of overlapping region vs. Field of View')
plt.grid()
plt.show()

In [ ]:
fish_length = 80
baseline = 13
working_distance = 100


fig, ax1 = plt.subplots(figsize=(20, 10))
mask = (results_df.working_distance == working_distance) & (results_df.fish_length == fish_length) & (results_df.baseline == baseline) & (~results_df.isnull().any(axis=1))
tdf = results_df[mask].sort_values('overlapping_region_size_horizontal', ascending=False).copy(deep=True)
ax1.plot(tdf.horizontal_field_of_view, tdf.overlapping_region_size_horizontal, 'b')
ax1.set_xlabel('Field of View (degrees)')
ax1.set_ylabel('Overlapping Region Size (cm)', color='b')
ax1.tick_params('y', colors='b')
ax2 = ax1.twinx()
ax2.set_ylabel('Resolution (lpmm)', color='r')
ax2.plot(tdf.horizontal_field_of_view, tdf.resolution, 'r')
ax2.tick_params('y', colors='r')
plt.grid()
plt.title('Overlapping Region Size and Resolution vs. Field of View')
plt.show()





In [ ]:
tdf[(tdf.overlapping_region_size_horizontal > 90) & (tdf.resolution > 1.5)]

In [ ]:
px_count_horizontal / (2 * camera_sensor_size_vertical * 10)

In [ ]:
n_a = 1.0
n_w = 1.33
f = 19.42
object_distances = np.arange(30, 500, 1)
image_distances = [n_a / (n_w / f - n_w / s) for s in object_distances]

plt.figure(figsize=(20, 10))
plt.plot(object_distances, image_distances)
plt.xlabel('Object Distance (cm)')
plt.ylabel('Virtual Image Distance (cm)')
plt.title('Object Distance vs. Virtual Image Distance')
plt.grid()
plt.show()

In [ ]:
virtual_image_distance_hash = {object_distance: image_distance for object_distance, image_distance in zip(object_distances, image_distances)}

In [ ]:
virtual_image_distance_hash[50]

In [ ]:
D = 75.0
x_s = 1.412
b = 12.7
l = 60
r_s = 120.0


In [ ]:
(D*x_s)/b - (1.0/b)*((D**2*l*x_s**2)/(b + l))**0.5

In [ ]:
f_values = list(np.arange(0.1, 2.5, 0.1))
field_of_view_values = [2 * np.arctan(x_s/2*f) for f in f_values]
resolution_values = [D * (x_s/b)] 

Pr = [((x_s*D/f - b - l)/(x_s*D/f - b)) * ((r_s*f)/D) for f in f_values]


In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(f_values, Pr)
plt.grid()
plt.show()

In [ ]:
(2*np.arctan(x_s / 2 * 1.0)) * (180 / np.pi)

In [ ]:
f = 1.2

In [ ]:
125 * (1.2 / (6.75 * 2.54))

In [ ]:
125 * (1.2 / (6.75 * 2.54))

In [ ]:
np.tan((60 / 2) * (np.pi/180))

In [ ]:
np.arctan(30 * (3.14159/180))